In [1]:
from ftplib import FTP
import pandas as pd
from os.path import join, exists
import os

In [2]:
targets=[
#     "Arabidopsis thaliana",
#     "Gossypium raimondii",
#     "Oryza sativa",
#     "Solanum lycopersicum",
#     "Solanum tuberosum",
    "Triticum aestivum",
    "Zea mays",
]
assembly_out = "assemblies"

In [3]:
def retrieve_assembly(ftp, df_index, target, assembly_out=""):
    print("Retrieving {} ...".format(target))
    row = df_index.loc[df_index["#name"]==target]
    target_folder = row["species"].to_string(index=False).strip()
    target_filename = row["#name"].to_string(index=False).strip() + "." + row["assembly"].to_string(index=False).strip()
    target_filename = target_filename.replace(" ", "_")
    target_filename += ".dna.toplevel.fa.gz"
    try:
        if not exists(join(assembly_out, target_filename)):
            ftp.cwd('fasta/' + target_folder + "/dna")
            ftp.retrbinary('RETR {}'.format(target_filename), open(join(assembly_out, target_filename), 'wb').write)
            ftp.cwd('../../..')
        print("Retrieving {} ... done".format(target))
    except:
        ftp.cwd('../../..')
        if exists(join(assembly_out, target_filename)):
            os.remove(join(assembly_out, target_filename))
        print("Retrieving {} ... failed - Filname {} not found".format(target, target_filename))
    

In [ ]:
# Make paths
if not exists(assembly_out):
    os.mkdir(assembly_out)

# Login to the ensembl ftp server
ftp = FTP('ftp.ensemblgenomes.org')
ftp.login()

'230 Login successful.'

In [ ]:
# Retrieve the latest list of plant species
ftp.cwd('pub/current/plants')
ftp.retrbinary('RETR species_EnsemblPlants.txt', open('species_EnsemblPlants.txt', 'wb').write)
df_index = pd.read_csv('species_EnsemblPlants.txt', sep='\t', header=0, index_col=False)
df_index.loc[df_index["#name"] == "Oryza sativa Japonica Group", "#name"] = "Oryza sativa"

In [ ]:
for target in targets:
    retrieve_assembly(ftp, df_index, target, assembly_out)

Retrieving Arabidopsis thaliana ...
Retrieving Arabidopsis thaliana ... done
Retrieving Gossypium raimondii ...
Retrieving Gossypium raimondii ... done
Retrieving Oryza sativa ...
Retrieving Oryza sativa ... done
Retrieving Solanum lycopersicum ...
Retrieving Solanum lycopersicum ... done
Retrieving Solanum tuberosum ...
Retrieving Solanum tuberosum ... done
Retrieving Triticum aestivum ...


In [ ]:
ftp.retrlines('LIST')